# BBLF AI Selector: Part 2: During Tournament Optimal Squad - Optimser (Round 2)

# 0. Prerequistes

In [1]:
# 0. Prerequistes

import pandas as pd
import numpy as np
import os
import random
from mip import Model, xsum, maximize, BINARY 

os.getcwd()
directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI'

# 1. Data Extraction 

In [7]:
# 1. Data Extraction 
    # Current Round & Optimal Number of Rounds
current_round = 2
opt_round = 3

    # Pull in player price data csv file 

price_df = pd.read_csv(os.path.join(directory,'data/python_datasets/player_price_round_2.csv'), low_memory=False)
print(price_df)

    # Player Role Flags

price_df["Wk_f"] = np.where((price_df["Role"] == "WK"), 1, 0)
price_df["Bat_f"] = np.where((price_df["Role"] == "WK") |(price_df["Role"] == "BAT") | (price_df["Role"] == "ALLR") , 1, 0)
price_df["Bowl_f"] = np.where((price_df["Role"] == "BOWL") | (price_df["Role"] == "ALLR") , 1, 0)
price_df = price_df[["Full_Name", "Price", "Team","Wk_f", "Bat_f", "Bowl_f", "Role", "Available", "In_Team"]].rename(columns = {"Full_Name": "Name"}) 
    
    # Pull in team fixture csv file

team_fix_df = pd.read_csv(os.path.join(directory,'data/python_datasets/team_loc_fixture.csv'), low_memory=False)
team_fix_df = team_fix_df[team_fix_df.Round >= current_round].dropna()
team_fix_df = team_fix_df[team_fix_df.Round <= current_round + opt_round - 1].dropna()
print(team_fix_df)

    # Pull player expected points csv file
exp_pts_df = pd.read_csv(os.path.join(directory,'data/python_datasets/bbl14_fullteam_model_score_during_rnd2.csv'), low_memory=False).drop(["Unnamed: 0", "player"], axis = 1)
print(exp_pts_df)

    # Join team fixture to player price to create a row for each game
game_df = pd.merge(price_df , team_fix_df, left_on = ["Team"], right_on = ["Team"], how = "left")
print(game_df)

    # Join on expected points for each game
print(game_df.columns)
print(exp_pts_df.columns)
player_df_raw = pd.merge(game_df, exp_pts_df, left_on = ["Name", "Team", "Opposition", "Venue", "Home_f"], right_on = ["Name", "Team", "opp", "venue", "Home_f"], how = "left")
player_df_raw["exp_pts"] = np.where((player_df_raw["Role"] == "WK"), player_df_raw["exp_pts"] + 10.51, player_df_raw["exp_pts"] + 4.53)
player_df_raw["weight"] = 1
print(player_df_raw)

    # Aggregate by player name (calculate total expected points for each player for x number of rounds)
player_df = player_df_raw.groupby(['Name', 'Price', "Team", "Wk_f", "Bat_f", "Bowl_f", "Role", "weight","Available", "In_Team"], as_index=False).agg(
exp_pts=('exp_pts',"sum"))
print(player_df)

      Last_Name First_Name First_Name_Short        Full_Name         player  \
0        Abbott       Sean               SA      Sean Abbott      SA Abbott   
1          Agar     Ashton               AC      Ashton Agar        AC Agar   
2          Agar        Wes               WA         Wes Agar        WA Agar   
3         Allen       Finn              NaN       Finn Allen     Finn Allen   
4         Allen     Fabian              NaN     Fabian Allen   Fabian Allen   
..          ...        ...              ...              ...            ...   
142       Wells        Jon               JW        Jon Wells       JW Wells   
143  Wildermuth       Jack               JD  Jack Wildermuth  JD Wildermuth   
144        Wood       Jack               JP        Jack Wood        JP Wood   
145      Wright        Mac                M       Mac Wright       M Wright   
146       Zampa       Adam                A       Adam Zampa        A Zampa   

      Price                 Team  Role  Available  

# 2. Optimisation Process (Round 2)

Optimisation Objective: Maximise the number of expected fantasy points

Constraints: 
1. Number of players selected must be 12
2. Atleast 1 wicketkeeper
3. Atleast 6 batters
4. Atleast 5 bowlers
5. Total budget of team is less than $1,762,800
6. Atleast 9 players in current team (3 trades)

In [8]:
# Optimisation Setup

points = player_df["exp_pts"]

price = player_df["Price"]

#model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"] = model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"].astype(object)

weight = player_df["weight"]
available = player_df["Available"]
in_team = player_df["In_Team"]
wk_weight = player_df["Wk_f"]
bat_weight = player_df["Bat_f"]
bowl_weight = player_df["Bowl_f"]

play_cnt, total_player = 12, range(len(price))
team_play_cnt, total_team_player = 9, range(len(price))
wk_cnt, total_wk = 1, range(len(price))
bat_cnt, total_bat = 6, range(len(price))
bowl_cnt, total_bowl = 5, range(len(price))
budget, total_budget = 1762800, range(len(price))

m = Model("knapsack")


x = [m.add_var(var_type=BINARY) for i in total_player]
print(x)

m.objective = maximize(xsum(points[i]*x[i] for i in total_player))

m += xsum(weight[i] * x[i] for i in total_player) == play_cnt
m += xsum(wk_weight[i] * x[i] for i in total_wk) >= wk_cnt
m += xsum(bat_weight[i] * x[i] for i in total_bat) >= bat_cnt
m += xsum(bowl_weight[i] * x[i] for i in total_bowl) >= bowl_cnt
m += xsum(price[i] * x[i] for i in total_budget) <= budget
m += xsum(available[i] * x[i] for i in total_player) == play_cnt
m += xsum(in_team[i] * x[i] for i in total_team_player) >= team_play_cnt

m.optimize()

selected = [i for i in total_player if x[i].x >= 0.99]
print("selected items: {}".format(selected))

sel_player_df = player_df.iloc[selected]

print("Total Expect Points:", sum(sel_player_df["exp_pts"]))
print("Total Team Cost:", sum(sel_player_df["Price"]))
print("Number of Wk:", sum(sel_player_df["Wk_f"]))
print("Number of Bat:", sum(sel_player_df["Bat_f"]))
print("Number of Bowl:", sum(sel_player_df["Bowl_f"]))
print("Available Players:", sum(sel_player_df["Available"]))
print("Current Players Remaining:", sum(sel_player_df["In_Team"]))

print(sel_player_df.sort_values(by = "exp_pts", ascending = False))

sel_player_df.to_csv('C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets/optimal_round_2.csv')


[<mip.entities.Var object at 0x000002329B0A5E40>, <mip.entities.Var object at 0x000002329B0A4D60>, <mip.entities.Var object at 0x000002329B0A5660>, <mip.entities.Var object at 0x000002329B0A52A0>, <mip.entities.Var object at 0x000002329B0A6740>, <mip.entities.Var object at 0x000002329B0A4310>, <mip.entities.Var object at 0x000002329B0A5300>, <mip.entities.Var object at 0x000002329B0A63E0>, <mip.entities.Var object at 0x000002329B0A4160>, <mip.entities.Var object at 0x000002329B0A4CA0>, <mip.entities.Var object at 0x000002329B0A5090>, <mip.entities.Var object at 0x000002329B0A6B60>, <mip.entities.Var object at 0x000002329B0A7220>, <mip.entities.Var object at 0x000002329B0A4820>, <mip.entities.Var object at 0x000002329B0A4F40>, <mip.entities.Var object at 0x000002329B0A4D00>, <mip.entities.Var object at 0x000002329B0A5030>, <mip.entities.Var object at 0x000002329B0A5420>, <mip.entities.Var object at 0x000002329B0A56C0>, <mip.entities.Var object at 0x000002329B0A5720>, <mip.entities.Var o

# Hayden Kerr Replacement

In [4]:
# 1. Data Extraction 
    # Current Round & Optimal Number of Rounds
current_round = 2
opt_round = 3

    # Pull in player price data csv file 

price_df = pd.read_csv(os.path.join(directory,'data/python_datasets/player_price_round_2_g3.csv'), low_memory=False)
print(price_df)

    # Player Role Flags

price_df["Wk_f"] = np.where((price_df["Role"] == "WK"), 1, 0)
price_df["Bat_f"] = np.where((price_df["Role"] == "WK") |(price_df["Role"] == "BAT") | (price_df["Role"] == "ALLR") , 1, 0)
price_df["Bowl_f"] = np.where((price_df["Role"] == "BOWL") | (price_df["Role"] == "ALLR") , 1, 0)
price_df = price_df[["Full_Name", "Price", "Team","Wk_f", "Bat_f", "Bowl_f", "Role", "Available", "In_Team"]].rename(columns = {"Full_Name": "Name"}) 
    
    # Pull in team fixture csv file

team_fix_df = pd.read_csv(os.path.join(directory,'data/python_datasets/team_loc_fixture.csv'), low_memory=False)
team_fix_df = team_fix_df[team_fix_df.Round >= current_round].dropna()
team_fix_df = team_fix_df[team_fix_df.Round <= current_round + opt_round - 1].dropna()
print(team_fix_df)

    # Pull player expected points csv file
exp_pts_df = pd.read_csv(os.path.join(directory,'data/python_datasets/bbl14_fullteam_model_score_during_rnd2.csv'), low_memory=False).drop(["Unnamed: 0", "player"], axis = 1)
print(exp_pts_df)

    # Join team fixture to player price to create a row for each game
game_df = pd.merge(price_df , team_fix_df, left_on = ["Team"], right_on = ["Team"], how = "left")
print(game_df)

    # Join on expected points for each game
print(game_df.columns)
print(exp_pts_df.columns)
player_df_raw = pd.merge(game_df, exp_pts_df, left_on = ["Name", "Team", "Opposition", "Venue", "Home_f"], right_on = ["Name", "Team", "opp", "venue", "Home_f"], how = "left")
player_df_raw["exp_pts"] = np.where((player_df_raw["Role"] == "WK"), player_df_raw["exp_pts"] + 10.51, player_df_raw["exp_pts"] + 4.53)
player_df_raw["weight"] = 1
print(player_df_raw)

    # Aggregate by player name (calculate total expected points for each player for x number of rounds)
player_df = player_df_raw.groupby(['Name', 'Price', "Team", "Wk_f", "Bat_f", "Bowl_f", "Role", "weight","Available", "In_Team"], as_index=False).agg(
exp_pts=('exp_pts',"sum"))
print(player_df)

      Last_Name First_Name First_Name_Short        Full_Name         player  \
0        Abbott       Sean               SA      Sean Abbott      SA Abbott   
1          Agar     Ashton               AC      Ashton Agar        AC Agar   
2          Agar        Wes               WA         Wes Agar        WA Agar   
3         Allen       Finn              NaN       Finn Allen     Finn Allen   
4         Allen     Fabian              NaN     Fabian Allen   Fabian Allen   
..          ...        ...              ...              ...            ...   
142       Wells        Jon               JW        Jon Wells       JW Wells   
143  Wildermuth       Jack               JD  Jack Wildermuth  JD Wildermuth   
144        Wood       Jack               JP        Jack Wood        JP Wood   
145      Wright        Mac                M       Mac Wright       M Wright   
146       Zampa       Adam                A       Adam Zampa        A Zampa   

      Price                 Team  Role  Available  

# 2. Optimisation Process (Round 2.2 - Hayden Kerr Replacement)

Optimisation Objective: Maximise the number of expected fantasy points

Constraints: 
1. Number of players selected must be 1
2. Atleast 0 wicketkeeper
3. Atleast 1 batters
4. Atleast 0 bowlers
5. Total budget of team is less than $176,200
6. Atleast 0 players in current team (New player)

In [6]:
# Optimisation Setup

points = player_df["exp_pts"]

price = player_df["Price"]

#model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"] = model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"].astype(object)

weight = player_df["weight"]
available = player_df["Available"]
in_team = player_df["In_Team"]
wk_weight = player_df["Wk_f"]
bat_weight = player_df["Bat_f"]
bowl_weight = player_df["Bowl_f"]

play_cnt, total_player = 1, range(len(price))
team_play_cnt, total_team_player = 0, range(len(price))
wk_cnt, total_wk = 0, range(len(price))
bat_cnt, total_bat = 1, range(len(price))
bowl_cnt, total_bowl = 0, range(len(price))
budget, total_budget = 176200, range(len(price))

m = Model("knapsack")


x = [m.add_var(var_type=BINARY) for i in total_player]
print(x)

m.objective = maximize(xsum(points[i]*x[i] for i in total_player))

m += xsum(weight[i] * x[i] for i in total_player) == play_cnt
m += xsum(wk_weight[i] * x[i] for i in total_wk) >= wk_cnt
m += xsum(bat_weight[i] * x[i] for i in total_bat) >= bat_cnt
m += xsum(bowl_weight[i] * x[i] for i in total_bowl) >= bowl_cnt
m += xsum(price[i] * x[i] for i in total_budget) <= budget
m += xsum(available[i] * x[i] for i in total_player) == play_cnt
m += xsum(in_team[i] * x[i] for i in total_team_player) == team_play_cnt

m.optimize()

selected = [i for i in total_player if x[i].x >= 0.99]
print("selected items: {}".format(selected))

sel_player_df = player_df.iloc[selected]

print("Total Expect Points:", sum(sel_player_df["exp_pts"]))
print("Total Team Cost:", sum(sel_player_df["Price"]))
print("Number of Wk:", sum(sel_player_df["Wk_f"]))
print("Number of Bat:", sum(sel_player_df["Bat_f"]))
print("Number of Bowl:", sum(sel_player_df["Bowl_f"]))
print("Available Players:", sum(sel_player_df["Available"]))
print("Current Players Remaining:", sum(sel_player_df["In_Team"]))

print(sel_player_df.sort_values(by = "exp_pts", ascending = False))

sel_player_df.to_csv('C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets/optimal_round_2_HK_repl.csv')


[<mip.entities.Var object at 0x000002329B0A41C0>, <mip.entities.Var object at 0x000002329B0A4280>, <mip.entities.Var object at 0x000002329B0A6B90>, <mip.entities.Var object at 0x000002329B0A6DD0>, <mip.entities.Var object at 0x000002329B0A52D0>, <mip.entities.Var object at 0x000002329B0A44F0>, <mip.entities.Var object at 0x000002329B0A5270>, <mip.entities.Var object at 0x000002329B0A5210>, <mip.entities.Var object at 0x000002329B0A51E0>, <mip.entities.Var object at 0x000002329B0A45E0>, <mip.entities.Var object at 0x000002329B0A50C0>, <mip.entities.Var object at 0x000002329B0A4D30>, <mip.entities.Var object at 0x000002329B0A4400>, <mip.entities.Var object at 0x000002329B0A4E80>, <mip.entities.Var object at 0x000002329B0A4FA0>, <mip.entities.Var object at 0x000002329B0A4EB0>, <mip.entities.Var object at 0x000002329B0A4DC0>, <mip.entities.Var object at 0x000002329B0A4E50>, <mip.entities.Var object at 0x000002329B0A4DF0>, <mip.entities.Var object at 0x000002329B0A4A90>, <mip.entities.Var o